**Process类**  
Process([group [, target [, name [, args [, kwargs]]]]])，由该类实例化得到的对象，表示一个子进程中的任务（尚未启动）

强调：
1. 需要使用关键字的方式来指定参数
2. args指定的为传给target函数的位置参数，是一个元组形式，必须有逗号

group参数未使用，值始终为None

target表示调用对象，即子进程要执行的任务

args表示调用对象的位置参数元组，args=(1,2,'hexin',)

kwargs表示调用对象的字典,kwargs={'name':'hexin','age':18}

name为子进程的名称  

**方法介绍**
p.start()：启动进程，并调用该子进程中的p.run() 
p.run():进程启动时运行的方法，正是它去调用target指定的函数，我们自定义类的类中一定要实现该方法  

p.terminate():强制终止进程p，不会进行任何清理操作，如果p创建了子进程，该子进程就成了僵尸进程，使用该方法需要特别小心这种情况。如果p还保存了一个锁那么也将不会被释放，进而导致死锁
p.is_alive():如果p仍然运行，返回True

p.join([timeout]):主线程等待p终止（强调：是主线程处于等的状态，而p是处于运行的状态）。timeout是可选的超时时间，需要强调的是，p.join只能join住start开启的进程，而不能join住run开启的进程

**属性介绍**  
p.daemon：默认值为False，如果设为True，代表p为后台运行的守护进程，当p的父进程终止时，p也随之终止，并且设定为True后，p不能创建自己的新进程，必须在p.start()之前设置

p.name:进程的名称

p.pid：进程的pid

p.exitcode:进程在运行时为None、如果为–N，表示被信号N结束(了解即可)

p.authkey:进程的身份验证键,默认是由os.urandom()随机生成的32字符的字符串。这个键的用途是为涉及网络连接的底层进程间通信提供安全性，这类连接只有在具有相同的身份验证键时才能成功（了解即可）

In [1]:
import time
import random
from multiprocessing import Process

创建并开启子进程  
**方法一**

In [15]:
def test(name):
    print("%s run" % name)
    time.sleep(random.randrange(0,2))
    print("%s end" % name)

p_obj = []
for i in range(0,4):
    p = Process(target=test, args=('%s' % i,))
    p_obj.append(p)
    p.start()
print('end')

0 run
1 run
end
2 run
3 run
3 end
0 end
1 end
2 end


**方法二**

In [24]:
class Test(Process):
    def __init__(self,name):
        super().__init__()
        self.name=name
        
    def run(self):
        print('%s run' % self.name)
        time.sleep(random.randrange(0,2))
        print('%s end' % self.name)
    
p_obj = []
for i in range(0,5):
    p = Test('%s' % i)
    p_obj.append(p)
    p.start()
print('end')

0 run
1 run
2 run
2 end
end
3 run
3 end
4 run
4 end
0 end
1 end


**run()**和**start()**的区别在于  
start()方法

开始线程活动。

对每一个线程对象来说它只能被调用一次，它安排对象在一个另外的单独线程中调用run()方法（而非当前所处线程）。

当该方法在同一个线程对象中被调用超过一次时，会引入RuntimeError(运行时错误)。

run()方法

代表了线程活动的方法。

你可以在子类中重写此方法。标准run()方法调用了传递给对象的构造函数的可调对象作为目标参数，如果有这样的参数的话，顺序和关键字参数分别从args和kargs取得。



In [32]:

import threading
 
class myThread(threading.Thread):
    def __init__(self, threadID, name, counter):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.counter = counter
 
    def run(self):
        currentTreadname = threading.currentThread()
        print("running in", currentTreadname)
        
thread = myThread(1,"mythrd",1)
thread.run()
thread.start()

running in <_MainThread(MainThread, started 4578622784)>
running in <myThread(mythrd, started 123145352839168)>


In [34]:
from multiprocessing import Process
import time
import random

class Piao(Process):
    def __init__(self,name):
        self.name=name
        super().__init__()
    def run(self):
        print('%s is piaoing' %self.name)
        time.sleep(random.randrange(1,3))
        print('%s is piao end' %self.name)


p=Piao('e')
p.daemon=True #一定要在p.start()前设置,设置p为守护进程,禁止p创建子进程,并且父进程死,p跟着一起死
p.start()
p.join() #等待p停止,等0.0001秒就不再等了
print('开始')

Piao-30 is piaoing
Piao-30 is piao end
开始


每启动一个新的进程相当创建全局变量的一份副本，子进程里的数据修改无法影响到主进程以及其他子进程中的数据，不同子进程之间的数据也不能共享，这是多进程与多线程最明显的区别  
所以下面都能成功

In [75]:
from multiprocessing import Process, Lock
import time
import random
import os

def work(lock):
    lock.acquire()
    global a
    if a > 0:
        print('%s success' % os.getpid(),a)
        a -= 1
        time.sleep(random.randrange(0,2))
    else:
        print('%s fail' % os.getpid())
    lock.release()

if __name__ == '__main__':
    a = 2
    lock = Lock()
    p_obj =[]
    for i in range(0,5):
        p = Process(target=work,args=(lock,))
        p.start()
        p_obj.append(p)


12883 success 2
12884 success 2
12885 success 2
12886 success 2
12887 success 2


多进程在读写文件的时候 加锁

In [79]:
from multiprocessing import Process,Lock
import json
import time
import random
import os

def work(filename,lock): #买票
    lock.acquire()
#     with lock:
    with open(filename,encoding='utf-8') as f:
        dic=json.loads(f.read())
        # print('剩余票数: %s' % dic['count'])
    if dic['count'] > 0:
        dic['count']-=1
        time.sleep(random.randint(1,2)) #模拟网络延迟
        with open(filename,'w',encoding='utf-8') as f:
            f.write(json.dumps(dic))
        print('%s 购票成功' %os.getpid())
    else:
        print('%s 购票失败' %os.getpid())
    lock.release()

if __name__ == '__main__':
    lock=Lock()
    p_l=[]
    for i in range(10):
        p=Process(target=work,args=('db.txt',lock))
        p_l.append(p)
        p.start()
    for p in p_l:
        p.join()

    print('主线程')

12937 购票成功
12938 购票成功
12939 购票成功
12940 购票失败
12941 购票失败
12942 购票失败
12943 购票失败
12944 购票失败
12945 购票失败
12946 购票失败
主线程


#### 信号量 `Semaphore`  
用法和锁类似 不过可以指定数量


In [92]:
from multiprocessing import Semaphore
def home(name, se):
    se.acquire()
    print('%s enter' % name)
    time.sleep(random.randint(1,3))
    print('%s leave' % name)
    se.release()
    
    
se = Semaphore(2)
p_obj = []
for i in range(0,5):
    p = Process(target=home, args=('%s'%i,se,))
    p.start()
    p_obj.append(p)
    

0 enter
1 enter
1 leave
2 enter
0 leave
3 enter
3 leave
4 enter
2 leave
4 leave


## 进程间通信

### 方式一 队列
队列先进先出，栈后进先出

创建队列的类（底层就是以管道和锁定的方式实现）
Queue([maxsize]):创建共享的进程队列，Queue是多进程安全的队列，可以使用Queue实现多进程之间的数据传递。  
maxsize是队列中允许最大项数，省略则无大小限制。


**方法**
q.put方法用以插入数据到队列中  
put方法还有两个可选参数：block和timeout。  
如果block为True（默认值），并且timeout为正值，该方法会阻塞timeout指定的时间，直到该队列有剩余的空间。  
如果超时，会抛出Queue.Full异常。如果block为False，但该Queue已满，会立即抛出Queue.Full异常。  

q.get方法可以从队列读取并且删除一个元素。   
get方法有两个可选参数：block和timeout。  
如果block为True（默认值），并且timeout为正值，那么在等待时间内没有取到任何元素，会抛出Queue.Empty异常。  
如果block为False，有两种情况存在，如果Queue有一个值可用，则立即返回该值，否则，如果队列为空，则立即抛出Queue.Empty异常  
 
q.get_nowait():同q.get(False)  
q.put_nowait():同q.put(False)  

q.empty():调用此方法时q为空则返回True，该结果不可靠，比如在返回True的过程中，如果队列中又加入了项目。  
q.full()：调用此方法时q已满则返回True，该结果不可靠，比如在返回True的过程中，如果队列中的项目被取走。  
q.qsize():返回队列中目前项目的正确数量，结果也不可靠，理由同q.empty()和q.full()一样  

### 各种队列的区别
Queue.Queue是进程内非阻塞队列。

multiprocess.Queue是跨进程通信队列。

多进程前者是各自私有，后者是各子进程共有。

In [11]:
from multiprocessing import Process,Queue
import time,random,os


def consumer(q):
    while True:
        time.sleep(random.randint(1,3))
        res=q.get()
        if res is None:break
        print('\033[45m消费者拿到了：%s\033[0m' %res)

def producer(seq,q):
    for item in seq:
        time.sleep(random.randint(1,3))
        print('\033[46m生产者生产了：%s\033[0m' %item)

        q.put(item)

if __name__ == '__main__':
    q=Queue()

    c=Process(target=consumer,args=(q,))
    c.start()

    producer(('包子%s' %i for i in range(5)),q)
    q.put(None)
    c.join()
    print('主线程')


消费者拿到了：包子0
生产者生产了：包子0
消费者拿到了：包子1
生产者生产了：包子1
生产者生产了：包子2
消费者拿到了：包子2
生产者生产了：包子3
消费者拿到了：包子3
消费者拿到了：包子4
生产者生产了：包子4
主线程


#### 创建队列的另一个类
JoinableQueue([maxsize])：这就像是一个Queue对象，但队列允许项目的使用者通知生成者项目已经被成功处理。通知进程是使用共享的信号和条件变量来实现的。

maxsize是队列中允许最大项数，省略则无大小限制。

 

JoinableQueue的实例p除了与Queue对象相同的方法之外还具有：

    q.task_done()：使用者使用此方法发出信号，表示q.get()的返回项目已经被处理。如果调用此方法的次数大于从队列中删除项目的数量，将引发ValueError异常
    q.join():生产者调用此方法进行阻塞，直到队列中所有的项目均被处理。阻塞将持续到队列中的每个项目均调用q.task_done（）方法为止


In [89]:
from multiprocessing import Process,JoinableQueue
import time,random
def consumer(q):
    while True:
        # time.sleep(random.randint(1,2))
        res=q.get()
        print('消费者拿到了 %s' %res)
        q.task_done()


def producer(seq,q):
    for item in seq:
        # time.sleep(random.randrange(1,2))
        q.put(item)
        print('生产者做好了 %s' %item)
    q.join()

if __name__ == '__main__':
    q=JoinableQueue()
    seq=('包子%s' %i for i in range(5))

    p=Process(target=consumer,args=(q,))
    p.daemon=True #设置为守护进程，在主线程停止时p也停止，但是不用担心，producer内调用q.join保证了consumer已经处理完队列中的所有元素
    p.start()
    producer(seq,q)

    print('主线程')


消费者拿到了 包子0
消费者拿到了 包子1
消费者拿到了 包子2
消费者拿到了 包子3
消费者拿到了 包子4
生产者做好了 包子0
生产者做好了 包子1
生产者做好了 包子2
生产者做好了 包子3
生产者做好了 包子4
主线程
